## 13.2 Decrease by half

Progressing towards the base case one item at a time is slow.
This and the next section show more efficient decrease-and-conquer algorithms
that decrease the input each time by a substantial amount.

One way is to decrease the input by a **constant factor** _f_ rather than by
a constant amount _a_, i.e. the size or value is reduced to _n_ / _f_
rather than _n_ – _a_. Usually _f_ = 2, i.e. the input is decreased by half.

### 13.2.1 Problem

Consider the [exponentiation operation](../02_Sequence/02_2_operations.ipynb#2.2.2-On-integers):
compute $b^{n}$ for integers _b_ (the base) and _n_ (the exponent),
with non-negative _n_.

A decrease-by-one definition is similar to the factorial:

- if _n_ = 0: $b^{n} = 1$
- if _n_ > 0: $b^{n} = b^{n-1} \times b$.

This effectively multiplies _b_ by itself _n_ times, in linear time.
The algorithm follows directly from the recursive definition,
so I skip to the code. For the mathematical notation,
it's convenient to have single-letter variable names, but
when writing code they should be descriptive.

In [1]:
def power_by_one(base: int, exponent: int) -> int:
    """Return the base to the power of the exponent.

    Preconditions: exponent >= 0
    """
    if exponent == 0:
        return 1
    else:
        return power_by_one(base, exponent - 1) * base

power_by_one(3, 20) == 3 ** 20     # use Python's operator to check

True

### 13.2.2 Algorithm

Using some properties of exponentiation,
we can halve the exponent instead of decreasing it by one,
to do fewer multiplications.
For example, 4⁶ = 4³ × 4³. Assuming 4³ requires three multiplications,
we need four instead of six multiplications to obtain 4⁶.
By halving the exponent we get twice the same expression
and only compute it once.
If the exponent is odd, we need one extra multiplication, e.g.
4⁵ = 4² × 4² × 4. The general recursive definition is:

- if _n_ = 0: $b^{n} = 1$
- if _n_ > 0 and is even: $b^{n} = b^{n/2} × b^{n/2}$
- if _n_ is odd: $b^{n} = b^{(n-1)/2} \times b^{(n-1)/2} \times b$.

The definition has one base case and two recurrence relations.
They cover all possible values of _n_.
For example, if _n_ = 1 then the last case applies and we have
*b*¹ = *b*⁰ × *b*⁰ × _b_ = 1 × 1 × _b_ = _b_.

Here's the algorithm, with an auxiliary variable for the subsolution
to avoid recomputing it.

1. if _n_ = 0:
   1. let _solution_ be 1
   2. stop
2. let _subsolution_ be power(_b_, floor(_n_ / 2))
2. if _n_ mod 2 = 0:
   1. let _solution_ be _subsolution_ × _subsolution_
2. otherwise:
   1. let _solution_ be _subsolution_ × _subsolution_ × _b_

The last steps could also be written as:

3. let _solution_ be _subsolution_ × _subsolution_
4. if _n_ mod 2 = 1:
   1. let _solution_ be _solution_ × _b_

I prefer the first alternative: its intent is clearer, in my opinion.

### 13.2.3 Complexity

Each recursive call takes constant time because it does at most four arithmetic
operations: integer division, modulo and one or two multiplications.
The complexity is therefore _r_ × Θ(1) = Θ(_r_),
where _r_ is the number of recursive calls.

Each extra recursive call can handle up to double the value of the exponent.
With _r_ recursive calls, the algorithm can handle any _n_ up to $2^r$.
You've seen this exponential growth rate
[before](../11_Search/11_5_subsets.ipynb#11.5.3-Complexity):
every item added to a set doubles the number of subsets the set has.

What we're really interested in is the inverse relationship:
how _r_ grows in terms of the input _n_.
The inverse of the exponential is the logarithm: log$_b$ _b_$^y$ = _y_
for any real number _b_ > 1.
The notation log$_b$ _n_ is read 'logarithm of _n_ to base _b_'.
For this problem, _n_ = 2$^r$, so log$_2$ _n_ = log$_2$ 2$^r$ = _r_. We say that
the exponentiation algorithm has **logarithmic complexity** Θ(log$_2$ _n_).

Actually, the base of the logarithm doesn't matter for complexity analysis
because it has been shown that the logarithms of the same number
in different bases only differ by a constant factor.
Thus, log$_a$ _n_ and log$_b$ _n_ have the same growth rate for
any bases _a_ and _b_, and we just write Θ(log _n_) without any base.

<div class="alert alert-info">
<strong>Info:</strong> Logarithms are covered in MU123 Unit&nbsp;13 Sections 4 and 5,
and in MST124 Unit&nbsp;3 Section&nbsp;4.
</div>

The safest way to analyse recursive algorithms is to write the
recursive definition of T and see which pattern it follows.
For this algorithm we have:

- if _n_ = 0: T(n) = Θ(1)
- if _n_ > 0: T(n) = T(floor(_n_ / 2)) + Θ(1).

Whether the algorithm halves an even exponent or halves and rounds down an
odd exponent makes no difference to the complexity, so we can simply write
T(_n_) = T(_n_ / 2) + Θ(1). It has been proven that such a
recursive definition leads to T(_n_) = Θ(log _n_).

<div class="alert alert-warning">
<strong>Note:</strong> If T(0) = Θ(1) and T(<em>n</em>) = T(<em>n</em> / 2) + Θ(1), then T(<em>n</em>) = Θ(log <em>n</em>).
</div>

When introducing [run-time measurements](../02_Sequence/02_8_time.ipynb#2.8.1-Checking-growth-rates),
I noted that although we assumed $b^{n}$ to take Θ(_n_)
to do _n_ constant-time multiplications,
Python's interpreter took less than linear time to compute it.
We henceforth assume exponentiation takes logarithmic time in _n_.

### 13.2.4 Code and performance

Let's implement the decrease-by-half approach.

In [2]:
def power_by_half(base: int, exponent: int) -> int:
    """Return the base to the power of the exponent.

    Preconditions: exponent >= 0
    """
    if exponent == 0:
        return 1
    subsolution = power_by_half(base, exponent // 2)
    if exponent % 2 == 0:
        return subsolution * subsolution
    else:
        return subsolution * subsolution * base

power_by_half(3, 20) == 3 ** 20

True

Since the complexity depends on the exponent only,
to measure the run-time I use always the same base,
start with a not too small exponent and double it each time.

In [3]:
exponent = 20
while exponent <= 200:
    %timeit -r 5 -n 10_000 power_by_one(3, exponent)
    exponent = 2 * exponent

3.27 µs ± 782 ns per loop (mean ± std. dev. of 5 runs, 10000 loops each)
8.04 µs ± 1.6 µs per loop (mean ± std. dev. of 5 runs, 10000 loops each)
13.4 µs ± 1.01 µs per loop (mean ± std. dev. of 5 runs, 10000 loops each)
27.7 µs ± 4.92 µs per loop (mean ± std. dev. of 5 runs, 10000 loops each)


The doubling of the run-time confirms the algorithm is linear in the exponent.
Now the decrease-by-half approach.

In [4]:
exponent = 20
while exponent <= 200:
    %timeit -r 5 -n 10_000 power_by_half(3, exponent)
    exponent = 2 * exponent

1.54 µs ± 181 ns per loop (mean ± std. dev. of 5 runs, 10000 loops each)
1.14 µs ± 222 ns per loop (mean ± std. dev. of 5 runs, 10000 loops each)
2.2 µs ± 116 ns per loop (mean ± std. dev. of 5 runs, 10000 loops each)
2.22 µs ± 379 ns per loop (mean ± std. dev. of 5 runs, 10000 loops each)


The run-time increases by about 200&nbsp;ns each time
because doubling the exponent requires a single extra multiplication.

<div class="alert alert-warning">
<strong>Note:</strong> If doubling the input size increases the run-time by a fixed amount,
then the complexity is logarithmic.
</div>

An exponential function with integer base greater than one grows very fast;
the logarithm function with the same base thus grows very slowly.
For example, $2^{20}$ is about one million, so computing $b^{1,000,000}$
takes just log$_2$ 1,000,000 ≈ 20 recursive calls!
Even if each one does two multiplications (the worst case),
40 multiplications is far better than doing a million of them.
The efficiency gain is tremendous, even compared to a linear algorithm.
If you find a logarithmic algorithm for a problem, you're on to a winner.

⟵ [Previous section](13_1_decrease_one.ipynb) | [Up](13-introduction.ipynb) | [Next section](13_3_variable_decrease.ipynb) ⟶